In [56]:
import retrieval_search
import json, os
from openai import OpenAI

In [50]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'subject': "Andes Festival"},
        boost_dict=boost,
        num_results=1
    )

    return results

In [31]:
def build_prompt(query, search_results):
    prompt_template = """
        Eres un asistente conversacional. Responde la PREGUNTA basado en el CONTEXTO de la base de datos.

        PREGUNTA: {question}

        CONTEXTO: 
        {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"seccion: {doc['section']}\npregunta: {doc['question']}\nrespuesta: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [32]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [40]:
def rag(query):
    search_results = search(query)
    print("Search results:", search_results)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [57]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

In [58]:
for subject_docs in docs_raw:
    for doc in subject_docs['documents']:
        doc['subject'] = subject_docs['subject']
        documents.append(doc)

In [59]:
index = retrieval_search.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["subject"]
)

index.fit(documents)

In [60]:
client = OpenAI()

In [63]:
print(rag("Cuando es el andes festival? Y a que hora?"))

Search results: [{'text': 'El Andes Festival se llevará a cabo el 10 de mayo de 2025.', 'section': 'Andes Festival', 'question': 'Cuando es el andes festival?', 'subject': 'Andes Festival'}]
Según el contexto proporcionado, el Andes Festival se llevará a cabo el 10 de mayo de 2025. Sin embargo, no se menciona la hora específica del evento en el contexto dado.
